<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LAI-and/or-$fAPAR$-extraction" data-toc-modified-id="LAI-and/or-$fAPAR$-extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LAI and/or $fAPAR$ extraction</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li><li><span><a href="#From-LAI-to-yield?" data-toc-modified-id="From-LAI-to-yield?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>From LAI to yield?</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li><li><span><a href="#Some-final-remarks" data-toc-modified-id="Some-final-remarks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Some final remarks</a></span><ul class="toc-item"><li><span><a href="#Noisy-signal" data-toc-modified-id="Noisy-signal-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Noisy signal</a></span></li><li><span><a href="#Maize-or...?" data-toc-modified-id="Maize-or...?-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Maize or...?</a></span></li><li><span><a href="#Complex-landscapes" data-toc-modified-id="Complex-landscapes-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Complex landscapes</a></span></li><li><span><a href="#The-resolution-issue" data-toc-modified-id="The-resolution-issue-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>The resolution issue</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import gdal
import pandas as pd
%matplotlib inline

from modis_funcs import *

/home/ucfajlg/.local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Leaf area index (LAI) from the MODIS sensors

The MODIS sensors have been acquiring data for the past two decades. The MODIS team has produced a large set of useful global land, atmosphere and ocean products, incluiding an LAI and fAPAR product. We have processed data over Ghana since 2002, and will now explore how the data looks like. We will be extracting area averaged estimates (in this case, we'll average over one of Ghana's 216 districts). We will also use a land cover product to select a particular land cover type (e.g. croplands, or forests). Our aim is to visualise and try to understand the dynamics of LAI over time. 

## LAI and/or $fAPAR$ extraction

We have downloaded and processed the [MODIS MCD15A2H product](https://lpdaac.usgs.gov/products/mcd15a2hv006/), which provides LAI and $fAPAR$ every 8 days from the combined observations of the TERRA and AQUA satellites. The product has been mosaicked to cover the whole of Ghana, and is available remotely. We have also collated the [MODIS MCD12Q1 product](https://lpdaac.usgs.gov/products/mcd12q1v006/) so you can select pixels that were classified as comin g from a particular land cover class.

The next cell gives you an easy GUI to extract the data. The data is extracted into a pandas dataframe, so you can easily save it to a CSV file for further playing around with it.


Be aware that running the data extraction GUI takes a bit! There's a lot of data to chomp through!!

### Exercise

Try selecting a particular district (e.g. Garu Tempane), and plotting

* LAI and fAPAR for years 2012 to 2017
* Change the Land cover from e.g. Woody Savannas to e.g. Croplands
* Try to spot patterns within years and land cover classes


Both plots and data are saved in the main folder (you can download them to your local disk). Feel free to experiment with them.




In [2]:
w = select_region_modis_lai()
w

interactive(children=(Dropdown(description='district', index=193, options=('Jomoro', 'Ellembelle', 'Nzema East…

You can easily plot all your attempts in here. Double click on this cell, and add the following Markdown code to plot graphs:
```
![](./Garu%20Tempane_Croplands_fAPAR.png)
```



## From LAI to yield?

In the absence of any information, we may think about ways of interpreting LAI and/or fAPAR. If we think of these two parameters are indicating the amount of leaves in the canopy, we can probably guess that a year where we have "fewer" leaves is a bad year, and one where we have "lots of leaves" is a good year. With this data, we can define a climatology and maybe look at some form of anomaly (e.g. a $z$ score). But we have data every (say) 8 days: what period do we choose? Obviously, we need to know when crops are likely to be in the ground. We could actually use the LAI to figure things out, but the data appears quite noisy. Let's just see whether we can see any differences by selecting the dates ourselves.


### Exercise

The next bit of code allows you to play around with the timing and plots the cummulative LAI as a function of day of year for the years you have extracted.

In [3]:
df = w.result
# or
# df = pd.read_csv("./Garu Tempane_Croplands_LAI.csv")
cummulative_lai_plots(df);

interactive(children=(SelectionRangeSlider(description='Sowing & Harvest', index=(0, 364), layout=Layout(width…

## Some final remarks

The examples above extract a great deal of data and visualise it, allowing you to maybe guess what's happening between different years. However, it is worth thinking about the following issues:

### Noisy signal

Based on your own experience, you may have an expectation of things like the amount of leaves in a crop to change slowly in a scale of days. Is this something you see in the plots above? (*hint*: No!!!! They're really noisy!) 

Think about what may be causing this. Remember the processing we've done:
1. For each date:
2. Do a spatial subset by district (e.g. select only pixels that fall within a given district).
2. Select all OK LAI/fAPAR retrievals (good quality).
3. Select only pixels that belong to a given land cover class
4. And where the percentage of the dominant class is assumed to be over some threshold
5. Aggregate all those pixels (mean, standard deviaton, percentiles, ...)

Think about the pixel size (~500 m nominally, but often close to 1.5km), and how the landscape looks like. Also think what e.g. "Croplands" means, and how different crops might behave.

### Maize or...?

Directed link the previous point. What if we are interested in monitoring maize, or sorghum, or cowpea? In general, different crops have different calendars, with different sowing, development and harvesting dates. What would we need to make an impact on this aspect?

### Complex landscapes

Small holder farms in Ghana also have tree crops (e.g. cocoa) in the middle of the fields. Will this affect our understanding of the LAI/fAPAR signal? How does the average size of the field with respect to the resolution cell affect things?

### The resolution issue

Clearly, the spatial resolution of MODIS might be an issue for monitoring small farms in Ghana. The very frequent overpasses (~twice a day) result in large pixels that *blur* the individual responses of different crops/fields. But we have higher spatial resolution sensors, albeit with a less frequent overpass, such as Sentinel 2.

The following notebook visualises some Sentinel 2 over the area around Tamale:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jgomezdans/demo_ghana/master)